In [1]:
import sys
sys.path.append('../')

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import sqlite3
import akshare as ak
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime
from dateutil.relativedelta import relativedelta
from tqdm import tqdm

from database_auto.db_data_downloader.downloader_base import DownloaderBase
import database_auto.database_config as db_config

pd.options.display.max_rows=None
pd.options.display.max_columns=None

!python --version

Python 3.8.10


In [2]:
import tensorflow as tf

# 只使用CPU进行训练
tf.config.set_visible_devices([], 'GPU')

# 打印Tensorflow版本
print(f"Tensorflow Version: {tf.__version__}")

# 检查是否有可用的GPU设备
if tf.test.is_built_with_cuda():
    print("TensorFlow GPU version is installed")
else:
    print("TensorFlow CPU version is installed")

# 检查TensorFlow是否能够访问GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print("GPU devices available:", gpus)
else:
    print("No GPU devices found. Running on CPU.")

# !nvidia-smi

2024-05-28 08:06:04.684345: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-28 08:06:05.645488: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Tensorflow Version: 2.13.1
TensorFlow GPU version is installed
GPU devices available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

mpl.rcParams['figure.figsize'] = (12, 10)
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# 绘图相关函数
def plot_series_dist(series):
    data = series
    plt.figure(figsize=(5,5))
    # 使用matplotlib画直方图
    plt.hist(data, bins=60, edgecolor='k', alpha=0.7)
    plt.xlabel('Value')
    plt.ylabel('Frequency')
    plt.title('Histogram of Data')
    plt.show()

def plot_metrics(history):
  metrics = ['loss', 'mean_squared_error']
  for n, metric in enumerate(metrics):
    name = metric.replace("_"," ").capitalize()
    plt.subplot(2,2,n+1)
    plt.plot(history.epoch, history.history[metric], color=colors[0], label='Train')
    plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[0], linestyle="--", label='Val')
    plt.xlabel('Epoch')
    plt.ylabel(name)
    plt.ylim([0, plt.ylim()[1]])
    plt.legend()

def plot_cm(true_labels, pred_labels):
    cm = confusion_matrix(true_labels, pred_labels)
    plt.figure(figsize=(5,5))
    sns.heatmap(cm, annot=True, fmt="g", cmap='Blues')
    plt.title('Confusion matrix')
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')

    accuracy = accuracy_score(true_labels, pred_labels)
    precision = precision_score(true_labels, pred_labels)
    recall = recall_score(true_labels, pred_labels)
    print(f"Accuracy: {accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}")

In [4]:
class PreProcessing:
    def __init__(self, db_downloader:DownloaderBase) -> None:
        self.db_downloader = db_downloader

    def _build_reg_label(self, stock_dataframe, N=15):
        stock_df = stock_dataframe.copy()
        # 计算未来N日内的最高收益率
        stock_df['max_return'] = stock_df['close'].rolling(window=N).max().shift(-N) / stock_df['close'] - 1
        # 计算未来N日内的最低收益率
        stock_df['min_return'] = stock_df['close'].rolling(window=N).min().shift(-N) / stock_df['close'] - 1
        # 计算未来N日内的收益率和（期望最高收益率越高越好，最低收益率也越高越好，由于最低收益率是负数，因此使用最高+最低来作为综合收益指标）
        stock_df['return_sum'] = stock_df['max_return'] + stock_df['min_return']
        stock_df['label'] = stock_df['return_sum']
        # 过滤第二天涨停股票
        stock_df = stock_df[stock_df['high'].shift(-1) != stock_df['low'].shift(-1)]
        return stock_df[['datetime', 'label']]

    def _process_one_stock(self, stock_code, start_date, end_date):
        stock_history = self.db_downloader._download_history_base_info(stock_code, start_date, end_date)
        stock_history = stock_history.replace("", np.NaN).ffill()
        stock_profile = self.db_downloader._download_all_stock_info(stock_code)
        stock_indicator = self.db_downloader._download_history_indicator_info(stock_code, start_date, end_date)
        stock_indicator = stock_indicator.replace("", np.NaN).ffill()
        stock_factor_date = self.db_downloader._download_history_date_factor_info(start_date, end_date)
        stock_factor_alpha158 = self.db_downloader._download_history_alpha158_factor_info(stock_code, start_date, end_date)
        stock_label = self._build_reg_label(stock_history)

        stock_df = stock_history.merge(stock_profile, on=['code'], how='left') \
                .merge(stock_indicator, on=['code', 'datetime'], how='left') \
                .merge(stock_factor_alpha158, on=['code', 'datetime'], how='left') \
                .merge(stock_factor_date, on=['datetime'], how='left') \
                .merge(stock_label, on=['datetime'], how='left')
        stock_df = stock_df.dropna()
        return stock_df

    def _get_index_hist_cons(self, benchmark, start_date, end_date):
        import baostock as bs
        bs.login()
        index_stock_cons_set = set()
        for cur_date in tqdm(pd.date_range(start=start_date, end=end_date, freq='B')):
            cur_date = datetime.strftime(cur_date, '%Y-%m-%d')
            if benchmark == '000016':
                # print("开始处理上证50...")
                dataframe = bs.query_sz50_stocks(date=cur_date).get_data()
            elif benchmark == '000300':
                # print("开始处理沪深300...")
                dataframe = bs.query_hs300_stocks(date=cur_date).get_data()
            elif benchmark == '000905':
                # print("开始处理中证500...")
                dataframe = bs.query_zz500_stocks(date=cur_date).get_data()
            if not dataframe.empty:
                index_stock_cons_set.update(
                    dataframe['code'].unique()
                )
        bs.logout()
        return index_stock_cons_set

    def _process_all_stock(self, benchmark, start_date, end_date):
        # 获取区间内benchmark的所有成份股
        # stock_code_list = self._get_index_hist_cons(benchmark, start_date, end_date)
        stock_code_list = list(ak.index_stock_cons(benchmark)['品种代码'].map(lambda x: 'sh.'+str(x)).unique()) # 获取指数成分股列表
        stock_df_list = []
        for stock_code in tqdm(stock_code_list, desc=f'Process: {benchmark} ...'):
            stock_df = self._process_one_stock(stock_code, start_date, end_date)
            if not stock_df.empty:
                stock_df_list.append(stock_df)
        return pd.concat(stock_df_list)

In [15]:
# import baostock as bs
# bs.login()
# bs.query_sz50_stocks(date='2023-01-01').get_data()

In [5]:
def get_rolling_data_period(backtest_start_date, backtest_duration=5, train_period=6, val_period=0.5, test_period=0.5):
    """
    Args:
        backtest_start_date (_type_): _description_
        backtest_duration (int, optional): _description_. Defaults to 5.
        train_period (int, optional): _description_. Defaults to 6.
        val_period (float, optional): _description_. Defaults to 0.5.
        test_period (float, optional): _description_. Defaults to 0.5.
    Returns:
        result: _description_
    """
    backtest_start_date = datetime.strptime(backtest_start_date, '%Y%m%d')
    backtest_end_date = backtest_start_date + relativedelta(years=backtest_duration) # 回测5年数据
    train_period = relativedelta(years=train_period) # 使用6年的训练数据
    val_period = relativedelta(months=(12 * val_period)) # 使用半年的验证数据
    test_period = relativedelta(months=(12 * test_period)) # 使用半年的测试数据(半年模型一更新)

    result = []
    rolling_flag = True
    bench_date = backtest_start_date
    while rolling_flag:
        if bench_date < backtest_end_date:
            test_start, test_end = bench_date, (bench_date + test_period - relativedelta(days=1))
            val_start, val_end = (test_start - relativedelta(days=1) - val_period), (test_start - relativedelta(days=1))
            train_start, train_end =(val_start - relativedelta(days=1) - train_period), (val_start - relativedelta(days=1))
            result.append({
                "train": [train_start.strftime("%Y%m%d"), train_end.strftime("%Y%m%d")],
                "val": [val_start.strftime("%Y%m%d"), val_end.strftime("%Y%m%d")],
                "test": [test_start.strftime("%Y%m%d"), test_end.strftime("%Y%m%d")]
            })
            bench_date += test_period
        else:
            rolling_flag = False 
    return result

In [6]:
def extract_train_val_data(df, train_start_date, train_end_date, val_start_date, val_end_date, test_start_date, test_end_date):
    train_start_date = pd.to_datetime(train_start_date)
    train_end_date = pd.to_datetime(train_end_date)
    val_start_date = pd.to_datetime(val_start_date)
    val_end_date = pd.to_datetime(val_end_date)
    test_start_date = pd.to_datetime(test_start_date)
    test_end_date = pd.to_datetime(test_end_date)

    train_data = df[(pd.to_datetime(df['datetime']) >= train_start_date) & (pd.to_datetime(df['datetime']) <= train_end_date)]
    val_data = df[(pd.to_datetime(df['datetime']) >= val_start_date) & (pd.to_datetime(df['datetime']) <= val_end_date)]
    test_data = df[(pd.to_datetime(df['datetime']) >= test_start_date) & (pd.to_datetime(df['datetime']) <= test_end_date)]

    print(f"train_data_size: {train_data.shape}")
    print(f"validation_data_size: {val_data.shape}")
    print(f"test_data_size: {test_data.shape}")
    return train_data, val_data, test_data

def df_to_dataset(dataframe, feature_cols, label_cols, shuffle=False, batch_size=32):
    features = dataframe[feature_cols]
    labels = dataframe[label_cols]
    ds = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=min(len(features), 10000), seed=1024)
    ds = ds.cache().batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler


class QuantileClipTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, lower_quantile=0.01, upper_quantile=0.99):
        self.lower_quantile = lower_quantile
        self.upper_quantile = upper_quantile
        
    def fit(self, X, y=None):
        # 计算给定分位数的分界值
        self.lower_bound_ = np.nanquantile(X, self.lower_quantile, axis=0)
        self.upper_bound_ = np.nanquantile(X, self.upper_quantile, axis=0)
        return self
    
    def transform(self, X, y=None):
        # 对整个数组应用剪辑操作
        return np.clip(X, self.lower_bound_, self.upper_bound_)

    def fit_transform(self, X, y=None, **fit_params):
        return self.fit(X, y).transform(X)

In [8]:
# 数据库初始化
db_conn = sqlite3.connect('../database_auto/hh_quant_auto.db')
db_downloader = DownloaderBase(db_conn, db_config)
proprocessor = PreProcessing(db_downloader=db_downloader)

# 相关配置
rolling_flag = False
# benchmark = '000905'
benchmark = '000016'  # 上证50
feature_config = {
    "target_features": ['label'],
    "numeric_features": ['turnover_rate', 'pe_ttm', 'ps_ttm', 'pcf_ncf_ttm', 'pb_mrq', 'KMID', 'KLEN', 'KMID2', 'KUP', 'KUP2', 'KLOW', 'KLOW2', 'KSFT', 'KSFT2', 'OPEN0', 'OPEN1', 'OPEN2', 'OPEN3', 'OPEN4', 'HIGH0', 'HIGH1', 'HIGH2', 'HIGH3', 'HIGH4', 'LOW0', 'LOW1', 'LOW2', 'LOW3', 'LOW4', 'CLOSE1', 'CLOSE2', 'CLOSE3', 'CLOSE4', 'VOLUME1', 'VOLUME2', 'VOLUME3', 'VOLUME4', 'ROC5', 'ROC10', 'ROC20', 'ROC30', 'ROC60', 'MAX5', 'MAX10', 'MAX20', 'MAX30', 'MAX60', 'MIN5', 'MIN10', 'MIN20', 'MIN30', 'MIN60', 'MA5', 'MA10', 'MA20', 'MA30', 'MA60', 'STD5', 'STD10', 'STD20', 'STD30', 'STD60', 'BETA5', 'BETA10', 'BETA20', 'BETA30', 'BETA60', 'RSQR5', 'RSQR10', 'RSQR20', 'RSQR30', 'RSQR60', 'RESI5', 'RESI10', 'RESI20', 'RESI30', 'RESI60', 'QTLU5', 'QTLU10', 'QTLU20', 'QTLU30', 'QTLU60', 'QTLD5', 'QTLD10', 'QTLD20', 'QTLD30', 'QTLD60', 'TSRANK5', 'TSRANK10', 'TSRANK20', 'TSRANK30', 'TSRANK60', 'RSV5', 'RSV10', 'RSV20', 'RSV30', 'RSV60', 'IMAX5', 'IMAX10', 'IMAX20', 'IMAX30', 'IMAX60', 'IMIN5', 'IMIN10', 'IMIN20', 'IMIN30', 'IMIN60', 'IMXD5', 'IMXD10', 'IMXD20', 'IMXD30', 'IMXD60', 'CORR5', 'CORR10', 'CORR20', 'CORR30', 'CORR60', 'CORD5', 'CORD10', 'CORD20', 'CORD30', 'CORD60', 'CNTP5', 'CNTP10', 'CNTP20', 'CNTP30', 'CNTP60', 'CNTN5', 'CNTN10', 'CNTN20', 'CNTN30', 'CNTN60', 'CNTD5', 'CNTD10', 'CNTD20', 'CNTD30', 'CNTD60', 'SUMP5', 'SUMP10', 'SUMP20', 'SUMP30', 'SUMP60', 'SUMN5', 'SUMN10', 'SUMN20', 'SUMN30', 'SUMN60', 'SUMD5', 'SUMD10', 'SUMD20', 'SUMD30', 'SUMD60', 'VMA5', 'VMA10', 'VMA20', 'VMA30', 'VMA60', 'VSTD5', 'VSTD10', 'VSTD20', 'VSTD30', 'VSTD60', 'WVMA5', 'WVMA10', 'WVMA20', 'WVMA30', 'WVMA60', 'VSUMP5', 'VSUMP10', 'VSUMP20', 'VSUMP30', 'VSUMP60', 'VSUMN5', 'VSUMN10', 'VSUMN20', 'VSUMN30', 'VSUMN60', 'VSUMD5', 'VSUMD10', 'VSUMD20', 'VSUMD30', 'VSUMD60'],
    "integer_categorical_features": ['month'],
    "string_categorical_features": ['industry', 'season'],
}
batch_size = 1024

# 是否开启滚动训练&回测
if rolling_flag:
    print("开启滚动回测...")
    backtest_period = get_rolling_data_period(
        backtest_start_date='20200101', # 回测开始日期
        backtest_duration=4, # 一共回测多久的数据（单位：年）
        train_period=6, # 使用过去多久的时间进行训练（单位：年）
        val_period=1, # 验证数据周期（单位：年）
        test_period=1, # 测试数据周期（单位：年）
    )
else:
    print("关闭滚动回测...")
    backtest_period = [
        {
            'train': ['2009-01-01', '2016-12-31'], # 8年训练
            'val': ['2017-01-01', '2018-12-31'], # 2年验证
            'test': ['2019-01-01', '2024-12-31'] # 5年预测
        }
    ]

backtest_period

关闭滚动回测...


[{'train': ['2009-01-01', '2016-12-31'],
  'val': ['2017-01-01', '2018-12-31'],
  'test': ['2019-01-01', '2024-12-31']}]

In [9]:
# df = proprocessor._process_all_stock('000016', '2024-01-01', '2024-12-31')
# df.tail()

In [10]:
date_period_params = backtest_period[0]
print(date_period_params)
train_start_date, train_end_date = date_period_params['train']
val_start_date, val_end_date = date_period_params['val']
test_start_date, test_end_date = date_period_params['test']
# 获取全区间数据
print("开始加载原始数据...")
df = proprocessor._process_all_stock(benchmark=benchmark, start_date=train_start_date, end_date=test_end_date)
# 抽取训练验证数据
print("开始拆分训练、验证、测试集合...")
train_data, val_data, test_data = extract_train_val_data(df, *[train_start_date, train_end_date, val_start_date, val_end_date, test_start_date, test_end_date])
# 从data中抽取相关特征数据
print("开始抽取特征数据...")
feature_columns = feature_config.get('numeric_features', []) + feature_config.get('integer_categorical_features', []) + feature_config.get('string_categorical_features', [])
label_columns = feature_config.get('target_features', [])
full_feature_columns = feature_columns + label_columns
train_df, val_df, test_df = train_data[full_feature_columns], val_data[full_feature_columns], test_data[full_feature_columns]
# 对相关特征进行特征工程
print("开始对特征进行预处理...")
feature_preprocess_pipeline = Pipeline(steps=[
    ('robust_scaler', RobustScaler()),
    ('minmax_scaler', MinMaxScaler()),
])
preprocess_feature_columns = feature_config.get('numeric_features', [])
train_df[preprocess_feature_columns] = feature_preprocess_pipeline.fit_transform(train_df[preprocess_feature_columns])
val_df[preprocess_feature_columns] = feature_preprocess_pipeline.transform(val_df[preprocess_feature_columns])
test_df[preprocess_feature_columns] = feature_preprocess_pipeline.transform(test_df[preprocess_feature_columns])

print("开始对标签进行预处理...")
label_preprocess_pipeline = Pipeline(steps=[
    ('quantile_clipper', QuantileClipTransformer()),
    ('robust_scaler', RobustScaler()),
    ('minmax_scaler', MinMaxScaler()),
])
preprocess_target_columns = feature_config.get('target_features', [])
train_df[preprocess_target_columns] = label_preprocess_pipeline.fit_transform(train_df[preprocess_target_columns])
val_df[preprocess_target_columns] = label_preprocess_pipeline.transform(val_df[preprocess_target_columns])
test_df[preprocess_target_columns] = label_preprocess_pipeline.transform(test_df[preprocess_target_columns])

{'train': ['2009-01-01', '2016-12-31'], 'val': ['2017-01-01', '2018-12-31'], 'test': ['2019-01-01', '2024-12-31']}
开始加载原始数据...
login success!


 17%|█▋        | 714/4174 [07:12<34:57,  1.65it/s]  


KeyboardInterrupt: 

In [ ]:
# 转换为tensorflow所使用的dataset
print("开始将DataFrame转换为DataSet...")
train_ds = df_to_dataset(train_df, feature_columns, label_columns, shuffle=True, batch_size=batch_size)
val_ds = df_to_dataset(val_df, feature_columns, label_columns, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test_df, feature_columns, label_columns, shuffle=False, batch_size=batch_size)  

开始将DataFrame转换为DataSet...


In [ ]:
# 准备模型训练
print("开始模型初始化 & 训练...")
from models.single_task.model_moe import QuantModel
model_config = {
        "seed": 1024,
        "feature_use_embedding": False,
        "feature_embedding_dims": 4,
        "numeric_features_with_boundaries": {k: pd.qcut(train_df[k], q=20, retbins=True, duplicates='drop')[1].tolist() for k in feature_config.get('numeric_features', [])},
        "integer_categorical_features_with_vocab": {k: list(train_df[k].unique()) for k in feature_config.get('integer_categorical_features', [])},
        "string_categorical_features_with_vocab": {k: list(train_df[k].unique()) for k in feature_config.get('string_categorical_features', [])},
    }
model = QuantModel(config=model_config)

initial_learning_rate = 5e-4

# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate,
#     decay_steps=5 * (len(train_df) // batch_size), # 每5个batch进行一次调整
#     decay_rate=0.9,
#     staircase=True)
model.compile(
    # optimizer=tf.keras.optimizers.Adam(lr_schedule),
    optimizer=tf.keras.optimizers.Adam(initial_learning_rate),
    # loss=tf.keras.losses.MeanSquaredError(),
    # loss=tf.keras.losses.MeanAbsoluteError(),
    loss=tf.keras.losses.Huber(),
    # metrics=[
    #     # tf.keras.metrics.MeanSquaredError(),
    #     # tf.keras.metrics.MeanAbsoluteError(),
    # ],
    )

EPOCHS = 500

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    verbose=1,
    patience=10,
    mode='min',
    restore_best_weights=True)

baseline_history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks = [early_stopping],
)

开始模型初始化 & 训练...
Epoch 1/500


2024-05-28 07:20:21.332814: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x150f002d29d0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2024-05-28 07:20:21.332964: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2024-05-28 07:20:21.358243: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-28 07:20:21.665276: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


50/50 [==============================] - 17s 203ms/step - loss: 0.3690 - val_loss: 0.1400
Epoch 2/500
50/50 [==============================] - 5s 103ms/step - loss: 0.2197 - val_loss: 0.0513
Epoch 3/500
50/50 [==============================] - 5s 95ms/step - loss: 0.1511 - val_loss: 0.0314
Epoch 4/500
50/50 [==============================] - 5s 106ms/step - loss: 0.1044 - val_loss: 0.0259
Epoch 5/500
50/50 [==============================] - 5s 107ms/step - loss: 0.0737 - val_loss: 0.0258
Epoch 6/500
50/50 [==============================] - 5s 92ms/step - loss: 0.0555 - val_loss: 0.0227
Epoch 7/500
50/50 [==============================] - 4s 89ms/step - loss: 0.0451 - val_loss: 0.0203
Epoch 8/500
50/50 [==============================] - 5s 99ms/step - loss: 0.0381 - val_loss: 0.0187
Epoch 9/500
50/50 [==============================] - 5s 101ms/step - loss: 0.0330 - val_loss: 0.0183
Epoch 10/500
50/50 [==============================] - 5s 97ms/step - loss: 0.0299 - val_loss: 0.0170
Epoch

In [ ]:
baseline_history.history.keys()

dict_keys(['loss', 'val_loss'])

In [ ]:
# plot_metrics(baseline_history)

In [ ]:
# model.save(f'./tf_models/{benchmark}/')

In [ ]:
test_data.head()

,code,datetime,open,high,low,close,volume,amount,turnover_rate,code_name,in_date,out_date,type,status,industry,pe_ttm,ps_ttm,pcf_ncf_ttm,pb_mrq,KMID,KLEN,KMID2,KUP,KUP2,KLOW,KLOW2,KSFT,KSFT2,OPEN0,OPEN1,OPEN2,OPEN3,OPEN4,HIGH0,HIGH1,HIGH2,HIGH3,HIGH4,LOW0,LOW1,LOW2,LOW3,LOW4,CLOSE0,CLOSE1,CLOSE2,CLOSE3,CLOSE4,VOLUME0,VOLUME1,VOLUME2,VOLUME3,VOLUME4,ROC5,ROC10,ROC20,ROC30,ROC60,MAX5,MAX10,MAX20,MAX30,MAX60,MIN5,MIN10,MIN20,MIN30,MIN60,MA5,MA10,MA20,MA30,MA60,STD5,STD10,STD20,STD30,STD60,BETA5,BETA10,BETA20,BETA30,BETA60,RSQR5,RSQR10,RSQR20,RSQR30,RSQR60,RESI5,RESI10,RESI20,RESI30,RESI60,QTLU5,QTLU10,QTLU20,QTLU30,QTLU60,QTLD5,QTLD10,QTLD20,QTLD30,QTLD60,TSRANK5,TSRANK10,TSRANK20,TSRANK30,TSRANK60,RSV5,RSV10,RSV20,RSV30,RSV60,IMAX5,IMAX10,IMAX20,IMAX30,IMAX60,IMIN5,IMIN10,IMIN20,IMIN30,IMIN60,IMXD5,IMXD10,IMXD20,IMXD30,IMXD60,CORR5,CORR10,CORR20,CORR30,CORR60,CORD5,CORD10,CORD20,CORD30,CORD60,CNTP5,CNTP10,CNTP20,CNTP30,CNTP60,CNTN5,CNTN10,CNTN20,CNTN30,CNTN60,CNTD5,CNTD10,CNTD20,CNTD30,CNTD60,SUMP5,SUMP10,SUMP20,SUMP30,SUMP60,SUMN5,SUMN10,SUMN20,SUMN30,SUMN60,SUMD5,SUMD10,SUMD20,SUMD30,SUMD60,VMA5,VMA10,VMA20,VMA30,VMA60,VSTD5,VSTD10,VSTD20,VSTD30,VSTD60,WVMA5,WVMA10,WVMA20,WVMA30,WVMA60,VSUMP5,VSUMP10,VSUMP20,VSUMP30,VSUMP60,VSUMN5,VSUMN10,VSUMN20,VSUMN30,VSUMN60,VSUMD5,VSUMD10,VSUMD20,VSUMD30,VSUMD60,weekday,day_of_week,day_of_month,month,season,label
60,sh.688981,2020-10-16,59.50,59.86,57.70,58.21,38030820.0,2.224178e+09,3.6560,中芯国际,2020-07-16,,股票,上市,电子,156.818263,17.920095,38.664818,9.457008,-0.021681,0.036303,-0.597222,0.006050,0.166667,0.008571,0.236111,-0.019160,-0.527778,1.022161,0.965470,0.984367,0.972342,0.883010,1.028346,1.028346,0.986944,0.991926,0.999313,0.991239,0.965470,0.958083,0.960831,0.879230,1.0,0.984367,0.961347,0.984195,0.986257,1.0,1.486745,0.759498,1.171918,2.198876,0.875107,0.936265,0.906717,1.154956,1.424498,1.028346,1.028346,1.028346,1.028346,1.030751,0.085896,0.171792,0.343584,0.515375,0.991239,0.983233,0.928517,0.946676,0.984768,1.144491,0.013888,0.061417,0.047926,0.084887,0.180822,0.002766,0.015706,-0.000789,-0.006342,-0.009381,0.099158,0.599459,0.009497,0.432588,0.820992,0.011235,0.000806,0.060824,0.107192,0.132262,0.989005,0.984745,0.984745,1.028586,1.314757,0.979625,0.859990,0.913039,0.928157,0.964920,5.0,10.0,20.0,23.0,23.0,0.969923,0.966907,0.958605,0.944742,0.221739,0.2,0.1,0.05,0.966667,0.800000,0.8,0.7,0.35,0.233333,0.116667,-0.6,-0.6,-0.3,0.733333,0.683333,0.401893,0.454797,0.328602,0.226705,0.650816,0.855797,0.284017,0.312852,0.044759,0.087912,0.6,0.5,0.50,0.400000,0.433333,0.4,0.5,0.50,0.600000,0.566667,0.2,0.0,0.0,-0.200000,-0.133333,0.857424,0.611680,0.603824,0.399375,0.382165,0.142576,0.388320,0.396176,0.600625,0.617835,0.714848,0.223359,0.207648,-0.201249,-0.235670,1.323407,1.076539,1.005835,1.140506,1.782309,0.556475,0.516126,0.419837,0.569235,1.233619,1.674397,1.762504,1.902590,1.932585,1.559979,0.525134,0.531553,0.498265,0.476898,0.326704,0.474866,0.468447,0.501735,0.523102,0.673296,0.050269,0.063107,-0.003470,-0.046203,-0.346592,4,Friday,16,10,Autumn,0.098780
61,sh.688981,2020-10-19,58.49,61.10,57.76,60.28,59006912.0,3.529811e+09,5.6725,中芯国际,2020-07-16,,股票,上市,电子,162.394861,18.557349,40.039773,9.793308,0.030604,0.057104,0.535928,0.014019,0.245509,0.012481,0.218563,0.029065,0.508982,0.970305,0.987060,0.932316,0.950564,0.938952,1.013603,0.993033,0.993033,0.953052,0.957863,0.958195,0.957200,0.932316,0.925182,0.927837,1.0,0.965660,0.950564,0.928334,0.950398,1.0,0.644515,0.958229,0.489507,0.755318,0.952389,0.890843,0.916224,1.107830,1.278368,1.013603,1.013603,1.013603,1.013603,1.013603,0.082946,0.165893,0.331785,0.497678,0.958195,0.958991,0.907548,0.918356,0.947357,1.100550,0.026509,0.067591,0.050109,0.077074,0.173629,0.013653,0.020419,0.000496,-0.004726,-0.008938,0.663146,0.836558,0.003433,0.291375,0.808278,0.013703,0.000567,0.076929,0.121169,0.163128,0.972528,0.955043,0.953318,0.980358,1.262376,0.945985,0.830458,0.881685,0.896284,0.931785,5.0,10.0,20.0,25.0,25.0,0.985383,0.983953,0.98

In [ ]:
# 输出回测预测
print("开始保存回测预测结果...")
model_pred_result = model.predict(test_ds)
output_df = test_data[['code', 'code_name', 'datetime']]
output_df['label'] = test_df['label']
output_df['label_pred'] = model_pred_result
output_df = output_df.rename(columns={
    'code': 'stock_code',
    'code_name': 'stock_name'
})
output_df.to_pickle(f'../../Offline/backtest/backtest_data/test/{benchmark}_{test_start_date}_回归任务_v4.pkl') 

开始保存回测预测结果...
59/59 [==============================] - 9s 116ms/step


In [ ]:
# output_df.head()